In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
from scipy import stats
import math
import yfinance as yf

In [17]:
# get tickers
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
# you can do either symbols.sort() or symbols = sorted(sp500...). Sorted will return a new list, while sort will directly modify the list.
symbols = sp500['Symbol'].tolist()
symbols.sort()
AllYF = yf.Tickers(' '.join(symbols)).tickers

To begin, we are going to do a simple model with only 1 value metric - price to earnings ratio (take their stock price and divide by how much earnings you expect them to have over the next year.)

In [16]:
# price to earnings ratio = price per share / earnings per share
# earnings per share = net income / end of period common shares outstanding
# common shares = shares you normally trade
# outstanding shares = shares currently owned by all shareholders (excluding treasury shares, which are owned by the company itself)
# pe ratio can have different calculations. This one is based on a current valuation, but yfinance has 2 others: trailing and forward.
# more information below

aapl = yf.Ticker("AAPL")
price = aapl.info['previousClose']
net_income = aapl.info['netIncomeToCommon']
outstanding_shares = aapl.info['sharesOutstanding']
eps = net_income / outstanding_shares
pe_ratio = price / eps
pe_ratio

33.335463728191

In [24]:
from concurrent.futures import ThreadPoolExecutor, as_completed

df_columns = ['Ticker', 'Stock Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

def fill_table(ticker, values):
    return [
        ticker,
        values.info.get('previousClose', None),
        values.info.get('forwardPE', None),
        'N/A'
    ]

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(fill_table, ticker, values) for ticker, values in AllYF.items()]
    results = [future.result() for future in as_completed(futures)]

results.sort(key=lambda x: x[0])

final_df = pd.DataFrame(data=results, columns=df_columns)
final_df.sort_values('Price-to-Earnings Ratio', ascending=True, inplace=True)

# We might not want to invest in companies that have negative earnings. 
final_df = final_df[final_df['Price-to-Earnings Ratio'] > 0]
final_df = final_df[:50]
final_df.reset_index(drop=True, inplace=True)

In [29]:
from CustomFunctions import get_int
portfolio_size = get_int()
position_size = portfolio_size / len(final_df.index)
for row in final_df.index:
    final_df.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_df.loc[row, 'Stock Price'])

# just show 10 entries as an example
final_df[:10]


,Ticker,Stock Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,UAL,46.76,4.081436,427
1,GM,44.15,4.436309,453
2,VTRS,11.90,4.459559,1680
3,EG,384.00,5.669529,52
4,AAL,10.43,5.693122,1917
5,F,11.01,5.765957,1816
6,DAL,43.03,5.897681,464
7,WBA,11.94,5.990148,1675
8,APA,30.78,6.478079,649
9,BWA,32.72,7.023810,611


Now, we are going to use multiple metrics to evaluate stocks. Here, we will filter for stocks with the lowest percentiles on:
- Price-to-earnings ratio
- Price-to-book ratio
- Price-to-sales ratio
- EV/EBITDA (Enterprise value / earnings before interest, taxes, depreciation, and amortization)
- EV/GP (Enterprise value / gross profit)

https://www.investopedia.com/investing/use-pe-ratio-and-peg-to-tell-stocks-future/

https://www.investopedia.com/terms/p/price-earningsratio.asp
#### More on PE Ratios
*with PE ratios, whether a ratio is high or low depends on the industry involved.*
- If PE ratio is high, it means the stock is overvalued. If it is low, it means the stock is undervalued.
- If PE ratio = 50, then it means investors are willing to pay $50 for every $1 of company earnings.
- Higher PEs are often taken to mean the firm has significant prospects for future growth.

There is also a **PEG ratio**, which is $\frac{\text{PE}}{\text{actual growth rate}}$. High PEG suggests PE is too high relative to growth.

Sometimes companies like recent startups will have negative earnings, so **Price to Sales Ratio** is used instead.
- This is $\frac{\text{Price per share}}{\text{Sales per share}}$

Another market value measure ratio is the **Market to Book Ratio**.
- This is $\frac{\text{Market value per share}}{\text{Book value per share}}$
- Book value per share is an accounting nuber, so it reflects historical costs. It sorta compares market value of the firm's investments to their cost.
    - Therefore, a value less than 1 could mean the firm hasn't been successful overall in creating value for its stockholders.

#### Enterprise Values
A company's enterprise value is an estimate of the market value of the company's operating assets (all assets except cash). These market values usually won't be available, so we can calculate this with:

${\text{Total market value of stock + Book value of all liabilities - Cash}}$

https://www.investopedia.com/ask/answers/06/amortizationvsdepreciation.asp

Given EBITDA = 
${\text{Earnings before Interest, Taxes, Depreciation, Amoritization}}$,
- h